In [13]:
import json
from pathlib import Path
import re
import shutil
import pandas as pd
import numpy as np
from IPython.display import HTML
import frontmatter

In [15]:
from pathlib import Path
import re
import shutil

def organize_md_files(directory):
    # 创建一个临时目录来存放分级整理的文件
    temp_dir = Path(directory).parent / 'temp'
    temp_dir.mkdir(parents=True, exist_ok=True)

    # 遍历指定目录下所有的md文件
    for md_file in Path(directory).glob('*.md'):
        # 获取文件名（不包括路径）
        file_name = md_file.name

        # 检查文件名是否符合 yyyy-mm-dd.md 格式
        if re.match(r'\d{4}-\d{2}-\d{2}\.md', file_name):
            # 提取年、月、日
            year, month, day = file_name.split('-')[:3]

            # 创建目标目录路径
            target_dir = temp_dir / year / month

            # 检查目标目录是否已经存在，如果不存在则创建
            if not target_dir.exists():
                target_dir.mkdir(parents=True, exist_ok=True)

            # 复制文件到目标目录
            target_path = target_dir / file_name
            if not target_path.exists():  # 确保目标路径不存在同名文件
                shutil.copy(md_file, target_path)
            else:
                print(f"File {file_name} already exists in {target_dir}, skipping.")

# 调用函数，指定要遍历的目录路径
organize_md_files('test')

In [ ]:
def increase_md_headers(directory):
    # 遍历指定目录及其子目录下所有的md文件
    for md_file in Path(directory).rglob('*.md'):
        # 读取文件内容
        with md_file.open('r', encoding='utf-8') as file:
            content = file.read()

        # 使用正则表达式增加每个标题的级别
        updated_content = re.sub(r'^(#+)', r'#\1', content, flags=re.MULTILINE)

        # 将更新后的内容写回文件
        with md_file.open('w', encoding='utf-8') as file:
            file.write(updated_content)
    print('all done.')

In [12]:
increase_md_headers('test')

all done.


In [17]:
df = pd.read_pickle('tidy_events.pkl.gz')
rep = df['ru']

def change_fn(d:str, rep):
    folder = Path(d)
    fns = folder.glob('*.md')
    for fn in fns:
        # print(f'{fn.stem}:{rep[str(fn.stem)]}')
        new_fn = fn.with_stem(rep[str(fn.stem)])
        fn.rename(new_fn)
    print('done!')

change_fn('src/pages/byday', rep)

done!


In [19]:
def df_to_json(df:pd.DataFrame, fn:str):
    """将带有日期的df转成多级json
    df.index: a str format data, eg '1969-04-21'
    to_dict make this as key.
    then break the key into year and month and put dates into
    """
    # 原始字典
    original_dict = df.to_dict('index')

    # 转换后的字典
    nested_dict = {}

    for date_key, inner_dict in original_dict.items():
        year, month, day = date_key.split('-')
        
        if year not in nested_dict:
            nested_dict[year] = {}
        
        if month not in nested_dict[year]:
            nested_dict[year][month] = {}
        
        nested_dict[year][month][date_key] = inner_dict

    # 将多级字典存储为 JSON 文件
    with open(fn, 'w') as json_file:
        json.dump(nested_dict, json_file, indent=4)

    print(f"多级字典已成功存储为{fn}文件")

In [9]:
rudf = pd.read_pickle('tidy_events.pkl.gz')
rudf.index = rudf['ru']

In [10]:
def add_fm(folder, df):
    data_dic = df.to_dict('index')

    folder = Path(folder)
    fs = folder.glob('*.md')
    for f in fs:
        post = frontmatter.load(f)
        meta = {}
        date = f.stem
        meta['days'] = data_dic[date]['days']
        meta['title'] = data_dic[date]['ru']
        meta['ru'] = data_dic[date]['ru']
        meta['ww'] = data_dic[date]['ww']
        meta['event'] = data_dic[date]['event']
        post.metadata = meta
        frontmatter.dump(post, f)
    print('done!')

add_fm('test', rudf)

done!


In [20]:
df_to_json(rudf, 'events.json')

多级字典已成功存储为events.json文件


In [16]:
add_fm('test', rudf)

done!


In [ ]:
def create_urls():
    htmls_d = Path('src/components/htmls')
    url_d = Path('src/pages/reports')
    url_d.mkdir(exist_ok=True)

    def make_report(out_d, stem):
        out_fn = out_d / f"{stem}.astro"  # 使用 f-string 简化文件名拼接

        report = f"""---
    import Report from '@/components/htmls/{stem}.html'
    import Layout from '@/layouts/Layout.astro'
    ---
    <Layout title='{stem}'>
        <Report />
    </Layout>

    """
        # print(f'report: {report}')  # 打印替换后的内容

        # 将修改后的内容写入新文件
        with open(out_fn, 'w', encoding='utf-8') as file:
            file.write(report)

    fs = htmls_d.glob('*.html')
    for f in fs:
        stem = str(f.stem)
        print(f'url:{stem}')
        make_report(url_d, stem)


In [ ]:
lable_css = """
<style>
    .mylab-group {
        display: flex;
        flex-flow: row wrap;
    }
    .mylab {
        display:flex;
        justify-content: center;
        border: 1px solid #ccc;
        border-radius: 5px;
        padding: 2px;
        margin: 2px;
        width: 5em;
        background-color: #f9f9f9;
    }
    .mylab p {
        color: #666;
        font-size: 0.8em;
        margin: 0;
    }
</style>
"""

def s_to_lable(s, title=None):
    """
    s可以为列表或者pd.Series
    """
    def create_lable(title):
        
        card_html = f"""
        <div class='mylab'>
            <p>{title}</p>
        </div>
        """
        return card_html

    def create_lable_group(cards_html):
        card_group_html = f"""
        <div class='mylab-group'>
            {cards_html}
        </div>
        """
        return card_group_html
    title_html = f'<h2>{title}</h2>\n' if title else ""
    labs = [create_lable(i) for i in s]
    labs_html = ''.join(labs)
    group_html = create_lable_group(labs_html) + lable_css
    display(HTML(title_html+group_html))
    # return group_html

In [ ]:
# test = pd.Series(range(10), name='num')
test = pd.Series(['中国银行']*10, name='num')
s_to_lable(test, 'this is title')

In [ ]:
s_to_lable(['中国银行']*10)